#  The inverse Ising problem

Areebah Anzar   

## Discussion

Previously, we studied the [Ising model](https://en.wikipedia.org/wiki/Ising_model), a simple model from statistical physics. The Ising model was originally applied to study ferromagnetism, but it has since been generalized and applied to understand a wide variety of physical and biological phenomena.

In our work on the Ising model, we wanted to understand how changing the parameters of the model affects behavior. In this homework problem, we will consider going in the opposite direction: given that we observe the behavior of an Ising system, how can we infer its parameters? In general, this kind of problem is called an [inverse problem](https://en.wikipedia.org/wiki/Inverse_problem). This problem specifically is referred to as the inverse Ising problem.

Inverse problems are very common in practical contexts, but they are also of great theoretical interest. See [here](https://en.wikipedia.org/wiki/Hearing_the_shape_of_a_drum) for a provocative example of a physics-inspired inverse problem.

## 1. Coupled spins

For this exercise we'll consider a very simple Ising system: two spins $\underline{\sigma}=\{\sigma_1, \sigma_2\}$, $\sigma_i\in\{\pm 1\}$ with a coupling constant $J$. We assume that the energy of interaction between each spin and an external magnetic field is $h$. The energy function is then

$$
E(\underline{\sigma}) = -h\sigma_1 -h\sigma_2 -J\sigma_1 \sigma_2\,.
$$

I've chosen some values for $h$ and $J$, gotten a set of sample configurations using Markov chain Monte Carlo, and saved them in a data file `configurations.csv`. We'll read in this data and analyze it, attempting to find back the original $h$ and $J$. In this section, we'll write a set of helper functions that we'll need to perform maximum likelihood inference.

### 1.a. Code the energy function

Code below to define a function, `compute_E`, that computes the energy of a configuration. Assume that the input `sigma` is an array of length two whose first element is $\sigma_1$ and whose second element is $\sigma_2$.

In [1]:
import numpy as np

def compute_E(sigma, h, J):
    """ 
    Returns the energy of a spin configuration.
    """
    #energy = -h*sigma1 -h*sigma2 - J*sigma1*sigma2
    energy = -h*sigma[0] - h*sigma[1] - J*sigma[0]*sigma[1]
    
    return  energy 

### 1.b. Code the partition function

In order to compute the probabilities of different configurations, we need to evaluate

$$
P(\underline{\sigma}) = \frac{e^{-E(\underline{\sigma})/T}}{Z}\,. 
$$

That means that we need to compute the partition function,

$$
Z = \sum_{\underline{\sigma}} e^{-E(\underline{\sigma})/T}\,.
$$

Fill in the code below to compute the partition function for a given set of $h$ and $J$ parameters. For simplicity, we'll normalize $T=1$.

In [2]:


def compute_Z(h, J):
    """ 
    Returns the partition function.
    """
    
    Z = 0 # initialize
    
    for sigma1 in [-1, 1]:
        for sigma2 in [-1, 1]:
            Z += np.exp(-compute_E([sigma1,sigma2],h,J) / 1)
    
    # Return the result
    
    return Z

### 1.c. Compute the derivative of the log-likelihood

As we have seen, the likelihood is defined as the probability of observing that data given a set of parameters

$$
\mathcal{L}(\{\underline{\sigma}^k\} | \{h, J\}) = \prod_{k=1}^{N} P_{h,J}(\underline{\sigma}^k)\,.
$$

Here $\underline{\sigma}^k$, $k=1, \ldots, N$ represent the $N$ configurations of the system that we've observed, and $P_{h,J}(\underline{\sigma})$ represents the probability of configuration $\underline{\sigma}$ in the Ising model with parameters $h$ and $J$. 

Below, write code to find the values of $h$ and $J$ that maximize the likelihood of the data. In order to do this, compute the derivative of the likelihood with respect to $h$ and $J$ so that we can use a gradient-based optimization algorithm to maximize it. 

This is easier numerically if we compute the gradient of minus the logarithm of the likelihood, $\ell$, instead. To do that, we take minus the logarithm of the above expression and compute derivatives with respect to $h$ and $J$. We will also divide by the number of observations $N$ for numerical stability. You can easily verify that

$$
\begin{align}
-\frac{d \ell}{N d h} &= -\frac{1}{N}\sum_{i=k}^{N} \left(\sigma^k_1 + \sigma^k_2\right) + \left\langle \sigma_1 + \sigma_2\right\rangle_{h, J}\,,\\
-\frac{d \ell}{N d J} &= -\frac{1}{N}\sum_{i=k}^{N} \left(\sigma^k_1 \times \sigma^k_2\right) + \left\langle \sigma_1 \times \sigma_2\right\rangle_{h, J}\,.
\end{align}
$$

To be clear, $\sigma^k_i$ represents the value of the $i$th spin in the $k$th configuration in the data. The average $\left\langle \cdot \right \rangle_{h, J}$ is an average over the Gibbs distribution for the Ising model with parameters $h$ and $J$,

$$
\left\langle f(\underline{\sigma}) \right\rangle = \sum_{\underline{\sigma}} f(\underline{\sigma}) P(\underline{\sigma})\,.
$$

It is now straightforward to compute these quantities using the functions that we've derived above. Fill in the code below to compute the gradient of minus the log-likelihood. We'll take `sigmas` to be a vector that holds the configuration data from the simulation. Each row represents a configuraton, i.e, a set of two Ising spin values. Similarly, `hJ` is also a vector whose first entry is the value of $h$ and whose second entry is the value of $J$.

In [3]:
def df(hJ, sigmas):
   
    
    # Initialize the gradient array
    
    grad = np.zeros(2)
    
    # Pre-compute the partition function
    
    Z = compute_Z(hJ[0], hJ[1])
    
    
    # The first entry is -dl/dh
    # First, get the contribution from the data
    
    for k in range(len(sigmas)):
        grad[0] += - (sigmas[k][0] + sigmas[k][1]) / float(len(sigmas)) 
    
    # Second, get the contribution from the average over the model
    
    for sigma1 in [-1, 1]:
        for sigma2 in [-1, 1]:
            grad[0] += (sigma1 + sigma2) * np.exp(-compute_E([sigma1, sigma2], hJ[0], hJ[1])) / Z
            
    
    # The second entry is -dl/dJ
    # First, get the contribution from the data
    
    for k in range(len(sigmas)):
        grad[1] += - (sigmas[k][0] * sigmas[k][1]) / float(len(sigmas))
        
    # Second, get the contribution from the average over the model
    
    for sigma1 in [-1, 1]:
        for sigma2 in [-1, 1]:
            grad[1] +=  (sigma1 * sigma2) * np.exp(-compute_E([sigma1, sigma2], hJ[0], hJ[1])) / Z 
    
    # Return the result
    
    return grad

## 2. Read in the data and code the optimization algorithm.

Use the steepest descent algorithm to find the parameters $h$ and $J$ that maximize the likelihood of the data. But first, we need to read in the data itself. It's stored in a `.csv` file, `configurations.csv`. You might want to look at the file directly. The first line of the file tells us what the values in each column mean. Each subsequent row is an observation, with the values in each columns separated by a comma. This is why this is called a `.csv` file: it stores **c**omma **s**eparated **v**alues.

### 2.a. Read in the data

Execute the code block below to read in the data and compute some summary statistics. 

In [4]:
# Read in the data

sigmas = np.loadtxt('configurations.csv', delimiter=',')


# Compute summary statistics

print('The average value of the spin at site 1 is %.3f' % np.mean(sigmas.T[0]))
print('The average value of the spin at site 2 is %.3f' % np.mean(sigmas.T[1]))
print('The average value of the sum of the spins at both sites is %.3f' % np.mean(sigmas))
print('The average value of the product of the spins is %.3f' % np.mean(sigmas.T[0]*sigmas.T[1]))

The average value of the spin at site 1 is 0.087
The average value of the spin at site 2 is 0.074
The average value of the sum of the spins at both sites is 0.081
The average value of the product of the spins is -0.181


### 2.b. Code the steepest descent algorithm

Fill in the code below to define a function `steepest_descent` that uses the steepest descent algorithm to find the minimum of minus the log-likelihood. The input to the function is the derivative function, `df`, the starting value, `x0`, and the data `sigmas`. In general the inverse Ising problem can be challenging, but this particular case should converge readily even if we choose a constant step size $t = 0.1$. 

In [5]:
def steepest_descent(df, x0, sigmas):
    """
    Run the steepest descent algorithm to find the minimum of the function whose gradient is df.
    The starting value for the function is x0.
    """
    
    # Initialize the system
    
    epsilon  = 0.0001  # Stopping condition -- end when |df/dx| < epsilon 
    max_iter = 100     # Maximum number of iterations
    it       = 0       # Current iteration
    
    x    = x0     # Current parameter value
    dfdx = df(x,sigmas) #  Initial value of the derivative
    
    # Report status
    print('iter\tx\tdf/dx')
    
    
    # Execute the steepest descent algorithm
    
    while np.sum(np.fabs(dfdx))>=epsilon and it<max_iter:
    
        # Report status
        print('{}\t{}\t{}'.format(it, x, dfdx))

        # Choose the step direction
        s =  -df(x,sigmas)

        # Choose how far to step in that direction
        t  = 0.1

        # Update the parameters
        x =  x+t*s# FILL THIS IN

        # Update the derivative
        dfdx =  df(x,sigmas) # FILL THIS IN

        # Update the iteration counter
        it += 1
        

    # Return the result
        
    return x

## 3. Find the parameters

Now that we've defined the steepest descent algorithm, we can now run it to find the underlying values of $h$ and $J$.

### 3.a. Run the steepest descent algorithm

Write code to run the steepest descent algorithm. We'll start at an initial guess of $h = J = 0$.

In [63]:
# Define the starting position

x0 =  [0,0]# FILL THIS IN


# Run steepest descent

x_min = steepest_descent(df, x0, sigmas) #  steepest_descent function to find the minimum


# Print the results

print('Found the minimum at r = {}'.format(x_min))

iter	x	df/dx
0	[0, 0]	[-0.161  0.181]
1	[ 0.0161 -0.0181]	[-0.12938534  0.16316106]
2	[ 0.02903853 -0.03441611]	[-0.10493506  0.14743927]
3	[ 0.03953204 -0.04916003]	[-0.08585296  0.13343704]
4	[ 0.04811734 -0.06250374]	[-0.07082909  0.12088057]
5	[ 0.05520025 -0.07459179]	[-0.05889978  0.10957096]
6	[ 0.06109022 -0.08554889]	[-0.04934987  0.09935601]
7	[ 0.06602521 -0.09548449]	[-0.04164409  0.09011373]
8	[ 0.07018962 -0.10449586]	[-0.03537855  0.08174253]
9	[ 0.07372747 -0.11267012]	[-0.0302461   0.07415542]
10	[ 0.07675208 -0.12008566]	[-0.02601149  0.06727634]
11	[ 0.07935323 -0.12681329]	[-0.02249324  0.06103786]
12	[ 0.08160256 -0.13291708]	[-0.01955042  0.05537966]
13	[ 0.0835576  -0.13845504]	[-0.01707289  0.05024744]
14	[ 0.08526489 -0.14347979]	[-0.01497405  0.04559216]
15	[ 0.08676229 -0.148039  ]	[-0.01318538  0.04136943]
16	[ 0.08808083 -0.15217594]	[-0.01165237  0.03753902]
17	[ 0.08924607 -0.15592985]	[-0.01033139  0.03406444]
18	[ 0.09027921 -0.15933629]	[-0.00918732  0